In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

2025-07-08 06:18:35.126082: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-08 06:18:35.129816: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-08 06:18:35.140004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751926715.156751   50267 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751926715.161527   50267 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751926715.174892   50267 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
mp_holistic = mp.solutions.holistic # For detection
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh 

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results =  model.process(image)  #Make Prediction
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image, results):
    if results.face_landmarks:
        mp_drawing.draw_landmarks(
            image, 
            results.face_landmarks, 
            mp_face_mesh.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(150, 150, 150), thickness=1, circle_radius=1),  # gray nodes
            mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=1)  # white mesh lines
        )

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=3),  # white joints
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=2)      # bright green lines
        )
    
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=3),    # red joints
            mp_drawing.DrawingSpec(color=(200, 100, 100), thickness=1, circle_radius=2) # muted red lines
        )
    
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 120, 255), thickness=2, circle_radius=3),  # blue joints
            mp_drawing.DrawingSpec(color=(100, 180, 255), thickness=1, circle_radius=2) # soft blue lines
        )


In [6]:
print("Checking for pose landmarks...")
print("pose_landmarks:", results.pose_landmarks)


Checking for pose landmarks...


NameError: name 'results' is not defined

In [7]:


len(results.face_landmarks.landmark)



NameError: name 'results' is not defined

In [8]:
def extract_keypoints(results):

    pose = (
        np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        if results.pose_landmarks
        else np.zeros(132)
    )
    
    face = (
        np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()
        if results.face_landmarks
        else np.zeros(1404)
    )
    
    left_hand = (
        np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
        if results.left_hand_landmarks
        else np.zeros(21 * 3)
    )
    
    right_hand = (
        np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
        if results.right_hand_landmarks
        else np.zeros(21 * 3)
    )

    return np.concatenate([pose, face, left_hand, right_hand])



In [9]:
extract_keypoints(results).shape

NameError: name 'results' is not defined

In [10]:
DATA_PATH = os.path.join('MP_Data') # Path for exported data -> numpy arrays

actions = np.array(['hello', 'thanks', 'iloveyou'])
no_sequences = 30
sequence_length = 30

In [ ]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [11]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #Read Feed
        ret, frame = cap.read() 

        #Detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        #Landmarks
        draw_styled_landmarks(image, results)

        print("📸 Frame OK")
        
        #Display
        cv2.imshow('OpenCV Feed', image)

        #Break
        if cv2.waitKey(10) & 0xFF == ord('q'): #Q key break
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751926755.965220   50400 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751926756.011000   50400 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751926756.017623   50405 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751926756.018778   50401 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751926756.019868   50396 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751926756.037042   50

<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionOutputs'>
📸 Frame OK
<class 'mediapipe.python.solution_base.SolutionO

In [12]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # Loop through actions
    for action in actions:
        # Loop through sequences (videos)
        for sequence in range(no_sequences):
            # Loop through video frames
            for frame_num in range(sequence_length):
                
                # Read Feed
                ret, frame = cap.read() 
        
                # Detections
                image, results = mediapipe_detection(frame, holistic)
                print(results)
        
                # Landmarks
                draw_styled_landmarks(image, results)

                # Wait Logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, f'Collection frame for {action} Video Number {sequence}', (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, f'Collection frame for {action} Video Number {sequence}', (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                # Display
                cv2.imshow('OpenCV Feed', image)

                # Save keypoints to .npy file
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                # Break
                if cv2.waitKey(10) & 0xFF == ord('q'):  # Q key break
                    break

# Release resources
cap.release()
cv2.destroyAllWindows()


W0000 00:00:1751926772.911075   50453 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751926772.942107   50453 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751926772.946675   50455 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751926772.946668   50460 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751926772.948411   50458 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751926772.958623   50455 inference_feedback_manager.cc:114] Feedback manager 

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}